# Introduction to Partial Differential Equations
---

## Chapter 3: Parabolic PDEs, the Heat Equation, and a Deep Dive into Fourier Series 
---

## Want to use Colab? [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec4.ipynb)

---

## Prepping the environment for interactive plots in Colab
---

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab - installing missing packages')
    !pip install ipympl
    from IPython.display import clear_output
    clear_output()
    exit()
else:
    print('Not running on CoLab - assuming environment has necessary packages')

In [ ]:
%matplotlib widget
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()

## Creative Commons License Information
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Partial Differential Equations: Theory and Computations</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" property="cc:attributionName" rel="cc:attributionURL">Troy Butler</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" rel="dct:source">https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations</a>.

In [ ]:
import numpy as np
%matplotlib widget 
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

---
## Section 3.4: Finite Difference Approximations
---

We again consider the initial boundary value problem (IBVP) introduced in [Section 3.2](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec2.ipynb) that models how the initial temperature in a rod of length $\ell$ evolves over time if there is no heat source over the length of the rod and the ends are kept at an ambient temperature (denoted by $0$ for simplicity), 

$$
\left\{\begin{array}{rcl}
    u_t &=& c u_{xx}, \ 0<x<\ell, t>0, \\
    u(0,t)=u(\ell,t) &=& 0, \ t>0, \\
    u(x,0) &=& f(x), \ 0<x<\ell.
\end{array}\right.
$$

Recall that $c>0$ is a thermal diffusivity parameter defining the "ease" by which heat is diffused throughout the rod. The function $f(x)$ denotes the initial temperature distribution.

In this notebook, we derive an explicit finite difference scheme to this IBVP.

For reference, we consider an IC $f$ and exact solution $u$ of the form given in the code cell below.

In [ ]:
# f is an IC and u_exact is an exact solution to the IBVP 

def f(x, ell):
    return np.sin(2*np.pi*x/ell)

def u_exact(x, t, c, ell):
    return np.exp(-4*np.pi**2*t * c/ell) * np.sin(2*np.pi*x/ell)

---
### Section 3.4.1:  The method of lines and a semidiscrete approximation
---

The [method of lines (MOL) discretization](https://en.wikipedia.org/wiki/Method_of_lines) is a popular way of initializing a discretization of many PDEs. From Wikipedia:

> The method of lines most often refers to the construction or analysis of numerical methods for partial differential equations that proceeds by first discretizing the spatial derivatives only and leaving the time variable continuous. 

<mark>**As a first step, we apply the "standard" finite difference scheme in space.**</mark>

Here, this means that we first discretize $[0,\ell]$ with $n+2$ evenly spaced points

$$
    x_j = j\Delta x, \ j=0,1,\ldots,n+1,
$$

where

$$
    \Delta x = \frac{\ell}{n+1}.
$$

In Chapter 2, with $\ell=1$, we used $h$ to denote this $\Delta x$ term. Why use $\Delta x$ here instead of $h$? The rationale of introducing new notation is to more easily distinguish between the discretization parameters associated with the discretization of the spatial dimension and the discretization of the time dimension (which is our second step). Compare this with the use of $\Delta x$ and $\Delta y$ in the discretization of the 2D Poisson equation of [Section 2.6](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2Sec6.ipynb). It is then perhaps obvious that we will use $\Delta t$ for the temporal discretization parameter, but we wait to perform that discretization. 

With this discretization in space, we subsequently approximate $u_{xx}$ by the finite difference scheme

$$
    u_{xx}(x_j,t) = \frac{u(x_{j-1},t)-2u(x_j,t)+u(x_{j+1},t)}{\Delta x^2} + {\cal O}(\Delta x^2).
$$

Substitution of this into the heat equation gives

$$
    u_t(x_j,t) = c\frac{u(x_{j-1},t)-2u(x_j,t)+u(x_{j+1},t)}{\Delta x^2} + {\cal O}(\Delta x^2).
$$

- Dropping the ${\cal O}(\Delta x^2)$ gives us the so-called MOL discretization, which is a ***semidiscrete*** approximation in the sense that we have discretized space but not time.

- Notice that the above equation defines a ***first-order system of ordinary differential equations*** of dimension $n$ (since we have Dirichlet conditions).

---
### Section 3.4.2: A fully discrete system
---

<mark>**The second step is to discretize in time.**</mark>

We now use a finite difference scheme to approximate $u_t(x_j,t)$ in the semidiscrete approximation, which means that we arrive at a ***fully discrete*** approximation.

- The choice of finite difference scheme we use to approximate $u_t$ is typically tied to a particular numerical method for solving ODEs based on the perspective of a MOL discretization.

- A one-sided difference that approximates $u_t$ will produce either the (explicit) forward or (implicit) backward Euler methods. Students may want to review [Section 1.3](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec3.ipynb).

- Initially, we use the forward Euler method since it gives an ***explicit*** method, which means it is easy to implement even though it is otherwise pretty awful.

---
#### Consider uniform time steps of length $\Delta t$
---

For each $m\in\mathbb{N}$, define

$$
    t_m := m\Delta t
$$

and

$$
    u_t(x_j,t_{m}) = \frac{u_t(x_j,t_{m+1})-u_t(x_j,t_m)}{\Delta t} + {\cal O}(\Delta t).
$$

If we plug this into the MOL discretization above, drop the ${\cal O}(\Delta t)$ (recall that we previously dropped a ${\cal O}(\Delta x^2)$ term to create the MOL discretization), and let $v_j^m$ denote the resulting approximation to $u(x_j,t_m)$, then we arrive at

$$
\large \boxed{    \frac{v_j^{m+1}-v_j^m}{\Delta t} = \frac{v_{j-1}^m - 2v_j^m+v_{j+1}^m}{\Delta x^2}, \ j=1,\ldots, n, \ m\geq 0.  }
$$

This now defines a ***fully discrete*** system of equations that we solve to obtain an approximation to $u(x_j, t_m)$ for $j=0,\ldots, n+1$ and $m\geq 0$.

---
#### The initial condition and the first time step
---

For $m=0$, which corresponds to $t=0$, we have that

$$
    v_j^0 = f(x_j), \ j=1,\ldots, n.
$$

Thus, for $m=0$, we use the initial condition in the above equation to determine $v_j^{1}$ for each $j=1,\ldots, n$.

***Note that we use the BCs to set $v_0^0=0$ and $v_{n+1}^0=0$.***

---
#### An **explicit** formula for $v_j^{m+1}$
---

We now solve for $v_j^{m+1}$ in the above discretization, where for $j=1,\ldots, n$, and $m\geq 0$, 

$$
\begin{align}
    \frac{v_j^{m+1}-v_j^m}{\Delta t} &= \frac{v_{j-1}^m - 2v_j^m+v_{j+1}^m}{\Delta x^2} \\ \\
    \Rightarrow v_j^{m+1} - v_j^m    &= \underbrace{\frac{c\Delta t}{\Delta x^2}}_{=:r} \left(v_{j-1}^m - 2v_j^m+v_{j+1}^m\right) \\ \\
    \Rightarrow v_j^{m+1} - v_j^m    &= rv_{j-1}^m - 2rv_j^m + rv_{j+1}^m \\ \\
    \Rightarrow v_j^{m+1}            &= rv_{j-1}^m + (1-2r)v_j^m + rv_{j+1}^m
\end{align}
$$

In summary, the scheme for obtaining $v_j^{m+1}$ is rewritten as

$$
    \boxed{v_j^{m+1} = rv_{j-1}^m + (1-2r)v_j^m + rv_{j+1}^m, \qquad j=1,\ldots, n, \qquad m\geq 0,}
$$

where $r=c\Delta t/\Delta x^2$. The ratio, $r$, of discretization parameters is ***key*** to the stability and accuracy of the scheme.

---
#### Stability requirements
---

We require

$$
    r \leq \frac{1}{2}
$$

in order to ensure ***stability*** in the numerical solution. We discuss this below and refer to the source text for more information.

Stability in the numerical solution is essentially stating that the numerical solution behaves ***qualitatively*** like the exact solution.
Since it makes no sense to ask how accurate a numerical solution is that behaves completely differently from the exact solution, we see that prior to establishing stability, it makes no sense to ask questions about accuracy.

In this new form, it should be pretty clear why we call this scheme ***explicit*** since computing the approximation at time $t_{m+1}$ only uses information available at time $t_m$.

In [ ]:
# An explicit method based on first form of the method

def explicit_heat_v1(n, f, r, T=1, ell=1, c=1):
    
    # Setup discretization
    x = np.linspace(0, ell, n+2)
    dx = x[1]-x[0]
    
    dt = r*dx**2 / c
    
    M = int(T/dt)  # The number of time steps to take M with a final time T and time step dt
    
    # The actual method is below
    
    # Setup the initial v using f and BCs
    v = f(x, ell)  # IC
    v[0], v[-1] = 0, 0  # Impose the BCs

    for i in range(1,M+1):
        v[1:-1] = r*v[0:-2] + (1-2*r)*v[1:-1] + r*v[2:]
        
    return v

In [ ]:
def plot_u_and_v(u, method, n, f, r, T=1, ell=1, c=1, fignum=0):
    # This function allows various methods for computing the
    # numerical solution to be passed to it as the method parameter.
    
    plt.figure(num=fignum)
    plt.clf()
    
    x = np.linspace(0, ell, n+2)
    
    plt.plot(x, u(x,T,c,ell), ls=':', c='b', label="exact $u$")
    
    v = method(n, f, r, T, ell, c)  # Use the method to compute numerical solution v
    
    plt.plot(x, v, ls='-.', c='r', label=r"v $\approx$ $u$")
    plt.legend()
    plt.show()

In [ ]:
%matplotlib widget

%reset -f out 

interact_manual(plot_u_and_v, 
            u = fixed(u_exact),
            method = fixed(explicit_heat_v1),
            n = widgets.IntSlider(value=5, min=5, max=100, step=1),
            f = fixed(f),
            r = widgets.BoundedFloatText(value=0.5, min=0.01, max=0.6, step=0.01),
            T = widgets.FloatSlider(value=0.1, min=0.05, max=0.5, step=0.01),
            ell = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1),
            c = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1),
            fignum = fixed(0))

---
#### A matrix-vector representation
---

The above scheme can also be rewritten using matrix-vector notation as

$$
    v^{m+1} = (I-rA)v^m.
$$

Here, $v^{m+1}$ and $v^m$ denote the $n$-dimensional vectors whose $j$th components are given by $v_j^{m+1}$ and $v_j^m$, respectively, and $A$ is the usual tri-diagonal matrix

$$
A = \begin{pmatrix}
        2 & -1 & 0 & \cdots & 0 \\
        -1 & 2 & -1 & \ddots & \vdots \\
        0 & \ddots & \ddots & \ddots & 0 \\
        \vdots & \ddots & -1 & 2 & -1 \\
        0 & \cdots & 0 & -1 & 2
    \end{pmatrix}
$$

The matrix-vector form is particularly useful for comparing to the implicit scheme we derive below.

In [ ]:
def make_A(n):
    A = np.zeros((n,n))
    np.fill_diagonal(A,2)
    A += np.diag(-np.ones(n-1),k=1)
    A += np.diag(-np.ones(n-1),k=-1)
    return A

In [ ]:
# An explicit method based on second matrix-vector form of the method

def explicit_heat_v2(n, f, r, T=1, ell=1, c=1):
    
    # Setup discretization
    x = np.linspace(0, ell, n+2)
    dx = x[1]-x[0]
    
    dt = r*dx**2 / c
    
    M = int(T/dt)  # The number of time steps to take M with a final time T and time step dt
    
    A = make_A(n)
    I = np.eye(n)
    
    # The actual method is below
    
    # Setup the initial v using f and BCs
    v = f(x, ell)  # IC
    v[0], v[-1] = 0, 0  # Impose the BCs

    for i in range(1,M+1):
        v[1:-1] = (I - r*A).dot(v[1:-1])
        
    return v

In [ ]:
%matplotlib widget

%reset -f out 

interact_manual(plot_u_and_v, 
            u = fixed(u_exact),
            method = fixed(explicit_heat_v2),
            n = widgets.IntSlider(value=5, min=5, max=100, step=1),
            f = fixed(f),
            r = widgets.BoundedFloatText(value=0.5, min=0.01, max=0.6, step=0.01),
            T = widgets.FloatSlider(value=0.1, min=0.05, max=0.5, step=0.01),
            ell = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1),
            c = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1),
            fignum = fixed(1))

---
### Section 3.4.2: An Implicit Scheme
---

If we return to the MOL discretization from above and instead choose to use the following finite difference scheme in time

$$
    u_t(x_j,t_{m+1}) = \frac{u(x_j,\color{red}{t_{m+1}})-u(x_j,t_m)}{\Delta t} + O(\Delta t),
$$

<mark>and also use values of $u$ at time $t_{m+1}$ in the spatial discretization</mark>, then we end up with an ***implicit scheme where we are using backward Euler in time.***

The scheme can be written as

$$
    \left( I + r A\right) v^{m+1} = v^m, \quad m\geq 0,
$$

where $A$ is the same matrix as above.

This may seem somewhat annoying since solving for $v^{m+1}$ requires inverting the matrix $I+\Delta t A$.

- However, we observe that this matrix is temporally invariant, so we can "invert once" and store the result to be used at each time step to compute $v^{m+1}$.

- Or, for large matrices, we never invert but rather apply the process of approximating solutions to the linear problem at each time step.

This method is ***unconditionally stable***, which means we can choose any discretization we wish and the behavior of the numerical solution to the heat equation is qualitatively correct (even if it is complete garbage in terms of accuracy).

In [ ]:
def implicit_heat(n, f, r, T=1, ell=1, c=1):
    
    # Setup discretization
    x = np.linspace(0, ell, n+2)
    dx = x[1]-x[0]
    
    dt = r*dx**2 / c
    
    M = int(T/dt)  # The number of time steps to take M with a final time T and time step dt
    
    # The actual method is below
    A = make_A(n)
    I = np.eye(n)
    discrete_soln_operator = np.linalg.inv(I+r*A)
    
    # Setup the initial v using f and BCs
    v = f(x, ell)  # IC
    v[0], v[-1] = 0, 0  # Impose the BCs

    for i in range(1,M+1):
        v[1:-1] = discrete_soln_operator.dot(v[1:-1])
        
    return v

In [ ]:
%matplotlib widget

%reset -f out 

interact_manual(plot_u_and_v, 
            u = fixed(u_exact),
            method = fixed(implicit_heat),
            n = widgets.IntSlider(value=5, min=5, max=100, step=1),
            f = fixed(f),
            r = widgets.BoundedFloatText(value=0.5, min=0.01, max=0.6, step=0.01),
            T = widgets.FloatSlider(value=0.1, min=0.05, max=0.5, step=0.01),
            ell = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1),
            c = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1),
            fignum = fixed(2))

---
## Navigation

- [Previous](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec3.ipynb)

- [Next](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec5.ipynb)

---